In [1]:
import pprint
import collections
import re
import random

import networkx

import wetsuite.helpers.meta
import wetsuite.helpers.patterns
import wetsuite.extras.gerechtcodes
import wetsuite.datasets

from importlib import reload

In [2]:
wetsuite.datasets.list_datasets()

bwb  = wetsuite.datasets.load('bwb-mostrecent-xml').data
cvdr = wetsuite.datasets.load('cvdr-mostrecent-xml').data
rsnl = wetsuite.datasets.load('rechtspraaknl-struc').data
# TODO: even more free-flowing text


### Create some out test data - ECLI

In [14]:
# Goes through Gbytes of text, will take a few minutes
reload(wetsuite.helpers.meta)
reload(wetsuite.helpers.patterns)

mostrefs     = 0
mostrefs_url = ''

with open('eclitest.txt','w') as eclitestfile:

    for store in ( #easier to skip one with comments this way
        bwb, 
        cvdr,
        rsnl,
    ):

        for i, (url, data) in enumerate( store.items() ):
            if isinstance(data, dict): # assume rechtspraak rather than the others
                data = str(data)  # close enough for now
            else: # assume it's a bytes object
                data = data.decode('utf8')
                
            if 1:
                # that regexp awkwardly tries to accept more, without too much nonsense
                matches = list( re.finditer(r'(?<!:)([eECc][a-zA-Z]+[:](?:[^\s]{1,10}[:]){2,7}[^\s]+)', data) ) #intentionally overaccepts at the end
                if len(matches) > 1:
                    #print( f'============== {url} == {len(matches)} matches ==============')
                    for match_object in matches:
                        txt = match_object.group(0)
                        if 'jci' in txt:
                            continue
                        #print( match_object , txt )
                        msg = f'{txt}\t{url}\n'
                        eclitestfile.write( msg )
                        #print( msg, end=''  )
                    eclitestfile.flush()

                #if i>20000:
                #    break            

            if 0: # count distinct sourceref types
                import wetsuite.helpers.koop_parse
                tree = wetsuite.helpers.etree.fromstring( data.encode('utf8') )

                try:
                    count_types = collections.defaultdict(int)
                    refs = wetsuite.helpers.koop_parse.cvdr_sourcerefs( tree )
                    typs = collections.defaultdict(int)
                    for ref in refs:
                        typs[ref[0]]+=1

                    if len(typs) > mostrefs:
                        mostrefs = len(typs)
                        mostrefs_url = url
                        print( 'New best: %s for %r'%(mostrefs, mostrefs_url) )
                except Exception as e:
                    print( e )
                    pass

                #if i>200:
                #    break

            if 0: # print all URLs with any extrefs
                if '<extref' in data:
                    print(url)
                    break

                #if i>200:
                #    break

            if 0: # test extraction of most identifier-type references in the text
                refs = wetsuite.helpers.patterns.find_identifier_references( data, nonidentifier=False ) 
                if len(refs)>0:
                    print( '================== %s =================='%url )
                    for ref in refs:
                        print( '  %s: %r'%(ref.pop('type'), ref.pop('text')) )
                        pprint.pprint(ref, indent=5)

                #if i>200:
                #    break

#### Get a subset of that ECLI test data 
(we don't need that many good examples)

In [ ]:
reload(wetsuite.helpers.meta)
reload(wetsuite.helpers.patterns)
reload(wetsuite.extras.gerechtcodes)

#count_str = collections.defaultdict(int)
#count_bad     = 0

good = []
bad  = []

with open('eclitest.txt','r') as eclitestfile:
    for line in eclitestfile:
        text, url = line.rstrip('\n').rsplit('\t', 1)
        
        try:
            parsed = wetsuite.helpers.meta.parse_ecli(text)
            #if parsed['country_code'].upper()=='NL':
            #    cc = parsed['court_code']
            #    if wetsuite.extras.gerechtcodes.case_insensitive_lookup(cc) is None:
            #        count_str[cc] += 1
            good.append( (text, url) )
        except ValueError as ve:
            bad.append( (text, url) )
            #count_bad += 1

            if 0:
                sve = str(ve)
                if 'separated by four colons' in sve:
                    continue
                if 'First' in sve:
                    continue
                if 'country' in sve:
                    continue
                if 'not look' in sve:
                    continue

            #count_str[text.split(':')[0]] += 1
            print(text, ve, url)

if 0:
    it = list( count_str.items() )
    it.sort(key=lambda x:x[1], reverse=True)
    for k, c in it:
        print('%5r %s'%(k, c))

print( len(good), len(bad) )

with open('good_ecli.txt','w') as good_ecli_file:
    for text, url in random.sample(good, 20000):
        good_ecli_file.write(f'{text}\t{url}\n')

with open('bad_ecli.txt','w') as bad_ecli_file:
    for text, url in random.sample(bad, 20000):
        bad_ecli_file.write(f'{text}\t{url}\n')

## 